# Module2: Python と scikit-learn でのモデル学習とチューニング
30分

このモジュールでは、Python と Scikit Learn を使用して機械学習のモデルを学習していきます。Azure Machine Leraning 上ではコンピューティングインスタンスとコンピューティングクラスタを使いモデルの学習ができます。

1. [Azure との接続](#Azure-との接続)
    1. [Notebooks から Azure Machine Learning を始める](#Notebooks-から-Azure-Machine-Learning-を始める)
    1. [Azure Machine Learning Python SDK](#Azure-Machine-Learning-Python-SDK)
    1. [ワークスペースに接続](#ワークスペースに接続)
    1. [ワークスペース内の Azure Machine Learning のリソースを確認する](#ワークスペース内の-Azure-Machine-Learning-のリソースを確認する)
1. [コンピューティングインスタンス上でモデル学習](#コンピューティングインスタンス上でモデル学習)
    1. [実験として学習スクリプトを実行する](#実験として学習スクリプトを実行する)
    1. [学習済みモデルを登録する](#学習済みモデルを登録する)
1. [コンピューティングクラスタ上で学習の並列実行](#コンピューティングクラスタ上で学習の並列実行)
    1. [計算リソースの作成](#計算リソースの作成)
    1. [ハイパーパラメータチューニングの実験を実行](#ハイパーパラメータチューニングの実験を実行)

# Azure との接続
## Notebooks から Azure Machine Learning を始める

Azure Machine Learning は機械学習ソリューションを作成、管理するためのクラウドベースのサービスです。
データサイエンティストが既存のデータ処理、モデル開発スキルやフレームワークを活用し、それらのワークロードをクラウドに拡張することをサポートするために設計されています。

多くのデータサイエンスや機械学習の作業がこのように Notebook で実現します。(Notebook の説明の割愛します。)


## Azure Machine Learning Python SDK
多くの Python コードが Notebook で実行でき、必要な Python パッケージがインストールされた環境で実行されます。今回の場合は、Azure Machine Learning コンピューティングインスタンス上の *Conda* 環境の中で Notebook を実行します。 この環境はデフォルトでインストールされており、データサイエンティストが普段の業務で利用する一般的な Python パッケージも含まれています。あわせて皆さんの Azure Machine Learning ワークスペース内のリソースを使用するための Azure Machine Learning Python SDK もインストールされています。

下記のセルを実行することで、**azureml-core** パッケージをインポートし、インストールされている SDK のバージョンを確認してみましょう。

In [ ]:
import azureml.core

print("Ready to use Azure ML", azureml.core.VERSION)

## ワークスペースに接続

全ての実験や関連するリソースはあなたの Azure Machine Learning ワークスペースの中で管理されています。作成済みのワークスペースに接続、あるいは Azure Machine Learning SDK を使って新しく作成することができます。

一般的にワークスペースとの接続情報を JSON ファイルとして保管しておくことをお勧めします。これによって、Azure subcription ID などの詳細を記憶しておく必要もなく容易に接続することが可能です。JSON 設定ファイルは、Azure ポータル上のワークスペースのブレード、あるいは Azure Machine Learning スタジオのワークスペース詳細ペインからダウンロードすることもできます。しかしもしワークスペース内のコンピューティングインスタンスを使用されている場合は、設定ファイルは既に *root* にダウンロードされています。

下記のコードでは設定ファイルを使ってワークスペースに接続します。

> **注釈**: Notebook のセッションで初めてワークスペースに接続する際は、Azure にサインインするために以下の手順を求められるかもしれません。`https://microsoft.com/devicelogin`　へのリンクをクリックすし、自動的に生成されたコードを入力することでAzure にサインインします。無事 Azure にサインインした後、開かれているタブを閉じてこの Notebook に戻れます。

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

!ls /config.json

## ワークスペース内の Azure Machine Learning のリソースを確認する

これでワークスペースと接続できましたので、リソースを使って作業をしてみましょう。例えば以下のコードを使ってワークスペース内のコンピューティングリソースを列挙することができます。

In [ ]:
print("Compute Resources:")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)

## 学習用のスクリプトを作成する

ここでは Python スクリプトを用いて糖尿病のデータセットを基に機械学習のモデルを学習します。それではスクリプトとデータ用のフォルダを作成して初めていきましょう。

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

学習用のスクリプトを作成しフォルダに保存する準備ができました。

今回は学習の実験をより柔軟に実行できるように実装します。パラーメータをスクリプトに与えることで、設定を変えて同じスクリプトの学習実験を繰り返し実行できるようになります。ここではロジスティック回帰モデルを学習するらいに、正則化(regularization) の割合を表すハイパーパラメータを、スクリプトのパラメータとして与えます。

> **注釈**: このコードはスクリプトを *作成* するものです。実行はされません！

In [ ]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# Set regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# load the diabetes dataset
print("Loading Data...")
# load the diabetes dataset
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

In [ ]:
# 出力されたスクリプトを確認
!head $training_folder/diabetes_training.py

# コンピューティングインスタンス上でモデル学習
## 実験として学習スクリプトを実行する

これで実験としてスクリプトを実行する準備が整いました。デフォルトの環境には **scikit-learn** のパッケージがインストールされておらず、明示的に設定する必要がありますのでご注意下さい。 *condad* 環境は初めて環境を実行する際にオンデマンドで構築され、以降の実行時には同じ設定でキャッシュされています。そのため初回実行時のみ時間がかかるはずです。

モデルのハイパーパラメータによって結果がどのように変化する確認できるよう、２種類の設定オブジェクトを準備しておきます。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed (we need pip, scikit-learn and Azure ML defaults)
packages = CondaDependencies.create(conda_packages=['pip', 'scikit-learn'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Create a script config1
script_config1 = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                arguments = ['--reg_rate', 0.1],
                                environment=sklearn_env) 

# Create a script config2
script_config2 = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                arguments = ['--reg_rate', 10],
                                environment=sklearn_env)


準備が整いましたので、実行用のオブジェクトを **Experiment** から生成し、以下のように実行します。

In [ ]:
# submit the experiment run
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config1)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion()

**Run** オブジェクトから指標や出力が取り出せます。

In [ ]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

それではもう一方の設定 *script_config2* を使って実行して比較してみましょう。

In [ ]:
# submit the experiment run
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config2)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion()

# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 学習済みモデルを登録する

実験の出力には学習済みモデルのファイル(**diabetes_model.pkl**) も含まれています。このモデルを Azure Machine Learning ワークスペースに登録することで、モデルのバージョンを追跡したり後から取り出すことも可能になります。

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## ハイパーパラメータのチューニング

多くの機械学習のアルゴリズムはハイパーパラメータ(学習に影響するパラメータ値だが、学習データ自体から推定しないもの) を必要とします。例えばロジスティック回帰モデルを学習する場合は *regularization rate* というハイパーパラメータを使ってモデルのバイアスを軽減することができます。あるいは畳み込みニューラルネットワーク(CNN) の例だと、*learning rate* と*batch size* によって重み付けやミニバッチで処理するデータのサイズを制御することができます。ハイパーパラメータの決定はモデル学習のパフォーマンス、モデルの学習時間に強く影響します。一般的には複数の候補の組み合わせを試すことで適切な設定値を探します。

今回は2つのハイパーパラメータを想定し分類モデルを学習します。しかし、この原理原則は Azure Machine Learning を用いたあらゆるモデル学習において適用することが可能です。また、より大規模なハイパーパラメータチューニングを想定し、コンピューティングクラスタを使った並列の実験を実行します。

In [ ]:
experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

それではモデルを学習するためのスクリプトを作成します。この例でいは、*Gradient Boosting* アルゴリズムを採用し、分類モデルを学習します。学習用のスクリプトには以下の要件を満たす必要があります。

- 最適化したいそれぞれのハイパーパラメータを引数に持つ(今回は*learning rate*とアルゴリズムの*estimator*の数です)
- 最適化するための評価指標をログとして残すように実装(今回は AUC と　accuracy の両方を記録し、モデル選定の際にどちらも考慮できるようにしています)

## データ準備

この実験では糖尿妙患者の詳細が含まれたデータセットを使います。下のセルを実行し、データセットを作成します。(既に存在している場合、既存のバージョンが使用されます)

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve

# Get the experiment run context
run = Run.get_context()

# Get script arguments
parser = argparse.ArgumentParser()

# Input dataset
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')

# Hyperparameters
parser.add_argument('--learning_rate', type=float, dest='learning_rate', default=0.1, help='learning rate')
parser.add_argument('--n_estimators', type=int, dest='n_estimators', default=100, help='number of estimators')

# Add arguments to args collection
args = parser.parse_args()

# Log Hyperparameter values
run.log('learning_rate',  np.float(args.learning_rate))
run.log('n_estimators',  np.int(args.n_estimators))

# load the diabetes dataset
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe() # Get the training data from the estimator input

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a Gradient Boosting classification model with the specified hyperparameters
print('Training a classification model')
model = GradientBoostingClassifier(learning_rate=args.learning_rate,
                                   n_estimators=args.n_estimators).fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the model in the run outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

# コンピューティングクラスタ上で学習の並列実行
## 計算リソースの作成

ハイパーパラメータのチューニングでは、異なる値の組み合わせによる複数の実験を繰り返し、パフォーマンス指標を比較します。効率的に達成するため、オンデマンドのクラウドコンピューティングの利点を活かしてクラスターを作成します。つまり複数の学習を並列実行させられます。

後述のコードで Azure Machine Learning コンピューティングクラスタを指定します。(まだ作成していなければ、新たに作成します)

> **重要**: 下記のコードを実行する前に、 *your-compute-cluster* を任意のコンピューティングクラスタ名に変更して下さい。クラスター名はグローバルでユニークな必要があり、文字数を2-16の範囲で指定してください。有効な文字は英数字とハイフンです。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster" #cc-ym210625
cluster_name = "cc-ym210625"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', max_nodes=10)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## ハイパーパラメータチューニングの実験を実行

Azure Machine Learning は*hyperdrive*と言うハイパーパラメータチューニングのための機能があります。この実験では子の実行が発行され、それぞれ異なるハイパーパラメータの組み合わせで実行されます。最良のモデルを生成した実行(任意のパフォーマンス指標によって選定される)は特定でき、登録やデプロイのために学習済みモデルが選択されます。
The run producing the best model (as determined by the logged target performance metric for which you want to optimize) can be identified, and its trained model selected for registration and deployment.

> **注釈**: この例では early stopping policy は明記しません。(後略。詳細は原文をご覧ください)

In [ ]:
#from azureml.core import Experiment, ScriptRunConfig, Environment
#from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
#from azureml.widgets import RunDetails

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed (we need scikit-learn, Azure ML defaults, and Azure ML dataprep)
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-dataprep[pandas]'])
sklearn_env.python.conda_dependencies = packages

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                # Add non-hyperparameter arguments -in this case, the training dataset
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=sklearn_env,
                                compute_target = training_cluster)

# Sample a range of parameter values
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--learning_rate': choice(0.01, 0.1, 1.0),
        '--n_estimators' : choice(10, 100)
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, # No early stopping policy
                          primary_metric_name='AUC', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6, # Restict the experiment to 6 iterations
                          max_concurrent_runs=6) # Run up to 2 iterations in parallel

# Run the experiment
experiment = Experiment(workspace=ws, name='mslearn-diabetes-hyperdrive')
run = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()